In [1]:
# importation les bibliotheque nécessaire
import numpy as np
import pandas as pd

In [ ]:
#!kaggle datasets download -d amanik000/kidney-disease-dataset

Dataset URL: https://www.kaggle.com/datasets/amanik000/kidney-disease-dataset
License(s): unknown




  0%|          | 0.00/1.94M [00:00<?, ?B/s]
100%|██████████| 1.94M/1.94M [00:00<?, ?B/s]


In [ ]:
import zipfile

with zipfile.ZipFile('../kidney-disease-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [4]:
import os
print(os.getcwd())

c:\Users\hadhr\Documents\chronic-kidney-disease\Chronic-Kidney-Disease\Notebooks


In [5]:
df = pd.read_csv("../data/kidney_disease_dataset.csv")
print(df.head())

   Age of the patient  Blood pressure (mm/Hg)  Specific gravity of urine  \
0                  54                     167                      1.023   
1                  42                     127                      1.023   
2                  38                     148                      1.016   
3                   7                      98                      1.017   
4                  67                     174                      1.015   

   Albumin in urine  Sugar in urine Red blood cells in urine  \
0                 1               4                   normal   
1                 3               2                   normal   
2                 0               0                 abnormal   
3                 4               0                 abnormal   
4                 1               1                   normal   

  Pus cells in urine Pus cell clumps in urine Bacteria in urine  \
0           abnormal              not present       not present   
1             normal    